In [5]:
import pyspark

try:
    type(sc)
except NameError:
    sc = pyspark.SparkContext('local[*]')

In [6]:
# reviewsRDD = reviewsRDD_.map(lambda review: review.split(","))
#reviewsRDD.take(2)

#data.first()
#test.take(2)

from stop_words import get_stop_words
import re

stop_words = get_stop_words('english')
stop_words = set(stop_words)


def quitar_reviews_repetidos(reviewsRDD):
    reviews_sin_rep = reviewsRDD.map( lambda review: (review.Text, review) )\
                    .reduceByKey( lambda review1, review2: review1 )

    reviews_sin_rep = reviews_sin_rep.map(lambda x: x[1])

    return reviews_sin_rep

def preprocesar(review):
    texto = quitar_signos(review.Text)
    resumen = quitar_signos(review.Summary)

    texto_palabras = texto.split(" ")
    resumen_palabras = resumen.split(" ")

    # Antes de quitar las stopWords armo los ngramas, sino
    # estos no tendrian sentido
    ngrams_text = ngrams(texto_palabras, 3)
    ngrams_resumen = ngrams(resumen_palabras, 3)

    text = quitar_stopWords(texto_palabras)
    resumen = quitar_stopWords(resumen_palabras)

    estructura = [review.Id, review.Prediction, resumen+text+ngrams_text+ngrams_resumen]

    return estructura


def quitar_signos(texto):
    #signos = ['(',')','<br />']
    #signos = r'\()\<br />'
    #return [review.Text.strip(signos)]
    text = ""
    if texto is not None:
        text = texto.replace("<br />", " ")
        text = re.sub("[^a-zA-Z' ]", '', text)

    #resumen = re.sub("[^a-zA-Z' ]", ' ', review.Summary)
    #'[\(\)\{\}<>]'
    #text_palabras = text.split(" ")
    #resumen_palabras = resumen.split(" ")
    return text

def ngrams(texto, n):
    #texto = texto.split(" ")
    lista_ngrams = []
    for i in xrange(0, len(texto)-n-1):
        lista_ngrams.append(" ".join(texto[i:i+n]))

    return lista_ngrams


def quitar_stopWords(texto):
    text = []
    for palabra in texto:
        if palabra.lower() not in stop_words:
            text.append(palabra)

    return text



In [7]:
import pyspark_csv as pycsv

#sc.addPyFile('/tmp/spark-3a633a6f-578e-41b5-a353-143f66408280/userFiles-dea1cdfe-5203-4304-9d70-0a839a83714b/pyspark_csv.py')

trainRDD = sc.textFile('data_train.csv')
train_dataframe = pycsv.csvToDataFrame(sqlCtx, trainRDD, parseDate=False)
train = train_dataframe.rdd

## CONTAR REVIEW REPETIDO
#train_sin_rep = train.map( lambda texto: (texto.ProductId + texto.UserId, 1) )\
#                    .reduceByKey( lambda a,b: a+b )
#print train_sin_rep.takeOrdered(10, lambda x: -x[1])

## QUITAR REVIEW REPETIDO
#train_sin_rep = train.map( lambda texto: (texto.ProductId + texto.UserId, texto) )\
#                    .reduceByKey( lambda texto1, texto2: texto1 )
#train_sin_rep = train_sin_rep.map(lambda x: x[1])


train_sin_rep = quitar_reviews_repetidos(train)

# VER SI HAY REVIEW REPETIDO
#a = train_sin_rep.map( lambda texto: (texto.Text, 1) )\
#                    .reduceByKey( lambda a,b: a+b )
#print "ordered:", a.takeOrdered(10, lambda x: -x[1])


train_preprocesado = train_sin_rep.map(preprocesar)

#for x in train_preprocesado.take(1):
#    for y in x:
#        print y



In [8]:
testRDD = sc.textFile('data_test.csv')
test_dataframe = pycsv.csvToDataFrame(sqlCtx, testRDD, parseDate=False)
test = test_dataframe.rdd

test_sin_rep = quitar_reviews_repetidos(test)

test_preprocesado = test_sin_rep.map(preprocesar)

for x in test_preprocesado.take(1):
    #print "x: ", x
    for y in x:
        print y


201541
3
[u'Entirely', u'Satisfied', u'Even', u'though', u'shipping', u'super', u'fast', u'good', u'hoped', u'pretty', u'sure', u'received', u'pound', u'half', u'', u'definitely', u'full', u'two', u'pounds', u'', u'guess', u'fine', u'see', u'finishing', u'given', u'flavor', u'decent', u'near', u'good', u'dried', u'papaya', u'can', u'seem', u'fresh', u'either', u'probably', u'wont', u'reordering', u'item', u'Even though the', u'though the shipping', u'the shipping was', u'shipping was super', u'was super fast', u'super fast it', u'fast it was', u'it was not', u'was not as', u'not as good', u'as good as', u'good as I', u'as I hoped', u'I hoped it', u'hoped it would', u'it would be', u"would be I'm", u"be I'm pretty", u"I'm pretty sure", u'pretty sure I', u'sure I only', u'I only received', u'only received a', u'received a pound', u'a pound and', u'pound and a', u'and a half', u'a half ', u'half  definitely', u' definitely not', u'definitely not the', u'not the full', u'the full two', u'f

In [9]:
def distancia(texto1_estruct, texto2_estruct):
    texto1 = set(texto1_estruct[2])
    texto2 = set(texto2_estruct[2])

    interseccion = len(texto1.intersection(texto2))
    union = len(texto1.union(texto2))

    return [texto2_estruct[0], texto2_estruct[1], float(interseccion)/float(union)]


def knn(texto_estruct):
    k = 10

    dist = train_preprocesado.map(lambda vecino: distancia(texto_estruct, vecino))
    k_mas_cercanos = dist.takeOrdered(k, key=lambda x: -x[2])

    return k_mas_cercanos

def predecir(k_mas_cercanos):
    prediccion_total = 0
    for cercano in k_mas_cercanos:
        prediccion_total += cercano[1]

    return float(prediccion_total) / len(k_mas_cercanos)


test = test_preprocesado.take(10)
#for t in test:
#    print "t: ", t

## PREDECIR VARIOS
for test_pred in test:
    print "TEST_PRED: ", test_pred[0:2]
    k_mas_cercanos = knn(test_pred)
    prediccion = predecir(k_mas_cercanos)

    print "PRED: ", prediccion


## PREDECIR UNO
"""
test_pred = test[1]
print "test: ", test_pred
print " "

k_mas_cercanos = knn(test_pred)
print k_mas_cercanos

prediccion = predecir(k_mas_cercanos)

print "PRED: ", prediccion
"""



TEST_PRED:  [201541, 3]
PRED:  3.0
TEST_PRED:  [6911, 2]
PRED:  3.8
TEST_PRED:  [461853, 5]
PRED:  4.6
TEST_PRED:  [563294, 5]
PRED:  5.0
TEST_PRED:  [471402, 5]
PRED:  4.6
TEST_PRED:  [139618, 5]
PRED:  4.8
TEST_PRED:  [248946, 5]
PRED:  4.9
TEST_PRED:  [163663, 1]
PRED:  3.8
TEST_PRED:  [140792, 5]
PRED:  4.9
TEST_PRED:  [340744, 1]
PRED:  2.1


'\ntest_pred = test[1]\nprint "test: ", test_pred\nprint " "\n\nk_mas_cercanos = knn(test_pred)\nprint k_mas_cercanos\n\nprediccion = predecir(k_mas_cercanos)\n\nprint "PRED: ", prediccion\n'